In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
from src.metrics import nearest_neighbor_distance, largest_cluster_fraction

box_size = 1.0
eps = 0.05

for i in range(5):
    positions = np.random.rand(100, 2) * box_size
    nn = nearest_neighbor_distance(positions, box_size=box_size)
    lcf = largest_cluster_fraction(positions, eps=eps, box_size=box_size)
    print(f"Run {i}: NN={nn:.3f}, LCF={lcf:.2f}")

Run 0: NN=0.052, LCF=0.05
Run 1: NN=0.053, LCF=0.04
Run 2: NN=0.055, LCF=0.04
Run 3: NN=0.053, LCF=0.04
Run 4: NN=0.053, LCF=0.06


## Test of clustering metrics

This notebook tests the basic clustering observables on a random
initial configuration (near-uniform distribution).


In [3]:
for i in range(5):
    positions = np.random.rand(100, 2) * box_size
    nn = nearest_neighbor_distance(positions, box_size=box_size)
    lc = largest_cluster_fraction(positions, eps=eps, box_size=box_size)
    print(f"Run {i}: NN={nn:.3f}, LCF={lc:.2f}")

Run 0: NN=0.052, LCF=0.05
Run 1: NN=0.047, LCF=0.10
Run 2: NN=0.050, LCF=0.05
Run 3: NN=0.048, LCF=0.07
Run 4: NN=0.050, LCF=0.07
